## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BBSBSBSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
953
+------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|            coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
| 2840.872029540232|-1025.2231097807996|-345.80720543419227|    412.1|BBSBSBSB| 47.3864| -92.8389|       SNWD|0.2353663159757168|0.13578293696097907| 0.1244532434741811|  0.270921

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USW00094931,1989.0,-1025.223110
1,USC00214652,1996.0,783.458386
2,USW00014918,1989.0,-879.984282
3,USC00218543,1975.0,-26.949997
4,USC00218543,1969.0,342.332914


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,CA006020559,CA006025203,CA006025205,CA006032119,CA006034075,CA006036904,USC00210059,USC00210515,USC00210520,USC00210643,...,USC00217460,USC00218543,USC00218618,USC00218621,USC00218700,USC00218763,USC00219059,USC00219173,USW00014918,USW00094931
year,,,,,,,,,,,,,,,,,,,,,
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379.492536,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-267.672193,NaN
1949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-631.805673,...,NaN,NaN,-657.055541,NaN,NaN,-195.287381,-280.933221,NaN,-396.982395,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455.833139,...,1065.490027,NaN,1443.947629,NaN,NaN,NaN,1015.733314,NaN,1170.047261,NaN
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-472.122958,NaN,219.677267,...,-232.848139,NaN,314.853196,NaN,NaN,297.738323,51.828738,NaN,-26.894272,NaN
1952.0,NaN,NaN,NaN,NaN,NaN,NaN,-573.669596,NaN,NaN,-541.564921,...,-283.664970,NaN,-802.369107,NaN,NaN,-67.647584,-692.764932,NaN,-570.166606,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,-642.653479,-221.754564,NaN,634.742811,...,-154.036566,NaN,750.281189,NaN,NaN,251.853724,941.669672,NaN,449.744332,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,-612.621530,-519.332870,NaN,-852.850814,...,-724.819228,NaN,-974.189487,NaN,NaN,-336.898829,-832.939832,NaN,-311.210730,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,1539.705690,1457.654520,NaN,1531.029160,...,1619.573851,NaN,1083.517848,NaN,NaN,1787.632842,1451.726394,NaN,1340.746030,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,-299.340905,-1253.749505,NaN,-1027.503271,...,58.554731,NaN,-287.312554,NaN,NaN,-314.922014,-228.154883,NaN,-82.068325,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  750.23614434
RMS removing mean-by-station=  732.635844818
RMS removing mean-by-year   =  396.294511574


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 750.23614434
0 after removing mean by year    = 396.294511574
0 after removing mean by stations= 362.368374542
1 after removing mean by year    = 359.504958806
1 after removing mean by stations= 358.962901172
2 after removing mean by year    = 358.77208431
2 after removing mean by stations= 358.680860668
3 after removing mean by year    = 358.632674085
3 after removing mean by stations= 358.606438396
4 after removing mean by year    = 358.592008355
4 after removing mean by stations= 358.584041122
